In [1]:
%%configure -f
{ "conf": { 
            "spark.jars.packages" : "io.delta:delta-core_2.11:0.6.1",
            "spark.sql.extensions" : "io.delta.sql.DeltaSparkSessionExtension",
            "spark.sql.catalog.spark_catalog" : "org.apache.spark.sql.delta.catalog.DeltaCatalog"
          }}

In [4]:
from delta import *

df = spark.read.format("csv").load("s3://acme-data-lake/cdc_data/test6.csv",header=True, inferSchema=True)
df.write.format("delta").mode("overwrite").save("s3://acme-data-lake/delta-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "s3://acme-data-lake/delta-table")
newData = spark.read.format("csv").load("s3://acme-data-lake/cdc_data/test7.csv",header=True, inferSchema=True)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.ProductID = newData.ProductID") \
  .whenMatchedUpdate(set = { "ProductID": col("newData.ProductID") }) \
  .whenNotMatchedInsert(values = { "ProductID": col("newData.ProductID") }) \
  .execute()

# test scenario for multiple updates on the same file/day

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1626357904766_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…